# Boosting for Classifier Decision Trees

## Goal

In this tutorial, we will demonstrate the use of an ensemble method known as *boosting* for
*classifier decision trees* using k-fold cross validation.

## Load libraries and data

First, we load the required libraries. We use:

- `rpart` package because it contains implementations of decision trees;
- `adabag` package because it has implemented the *boosting* algorithm; and
- `caret` package to control trina-validation-test data.


In [ ]:
library(rpart)
library(adabag)
library(caret)


To load an `*.csv` file, simply use the function `read.csv()`. In addition, use `factor()`
to convert our response variable of interest to `factor`.


In [ ]:
hmeq <- read.csv(file.path("data", "hmeq.csv"), header = TRUE)
hmeq$BAD <- factor(hmeq$BAD)
head(hmeq)


## Create train-test split



In [ ]:
set.seed(12345)
train_idx <- createDataPartition(hmeq$BAD, p = 0.7, list = FALSE)
train_data <- hmeq[train_idx, ]
test_data  <- hmeq[-train_idx, ]


In [ ]:
head(train_data)



In [ ]:
head(test_data)



## Perform cross-validation to compare models

Let's split the train data into ten groups using `createFolds` from `caret` package:


Evaluate the performance of the models using 10-fold cross-validation:



Errors for the decision tree model:



Errors for boosting model:



Compare the mean error:

